# Import Libraries

In [ ]:
!pip install pip install tensorflow==2.7.0
!pip install tensorflow-quantum==0.7.2
!pip install pennylane

In [ ]:
!pip install cirq==0.13.1

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re

import pennylane as qml
from pennylane import numpy as p_np

from pennylane.templates.state_preparations import MottonenStatePreparation
from pennylane.templates.layers import StronglyEntanglingLayers
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# load our data
data = pd.read_csv("tweet_emotions.csv")
plt.style.use('fivethirtyeight')

# Data Visualization

In [ ]:
sns.countplot(y=data["sentiment"])

# Data Cleanup and Prepare

In [ ]:
stop_words = set(stopwords.words('english'))
from nltk.stem import RegexpStemmer
snowball = RegexpStemmer('ing$|s$|e$|able$', min=4)

data["content"] = data["content"].apply(lambda x: " ".join([snowball.stem(re.sub(r'[^\w\d]', "", word)) for word in x.lower().split() if word not in stop_words]))

X = data["content"]
tokenize = Tokenizer()

tokenize.fit_on_texts(X)

max_length = 64
vocab_size = len(tokenize.word_index) + 1
X = pad_sequences(tokenize.texts_to_sequences(X), maxlen=max_length, padding="post")

# Preparation of Quantum5

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights": (3, n_qubits, 3)}

qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)
clayer1 = tf.keras.layers.Dense(2)
clayer2 = tf.keras.layers.Dense(13, activation="softmax")
model = tf.keras.models.Sequential([clayer1, qlayer, clayer2])
opt = tf.keras.optimizers.SGD(learning_rate=0.8)
model.compile(opt, loss='mae')

# Preparation of Label

In [ ]:
Y = data['sentiment']
from sklearn.preprocessing import LabelEncoder

func = LabelEncoder()

Y = func.fit_transform(Y)
Y = tf.convert_to_tensor(Y)

X = tf.constant(X)
Y = tf.one_hot(Y, depth=1)

# Model Fitting

In [ ]:
model.fit(X,Y, epochs=1, batch_size=256, shuffle=True,steps_per_epoch=1,validation_split=0.1)

# Preparation of Prediction Data and Prediction Equation

In [ ]:
a = "Yes, I am very normal!"
tokenize.fit_on_texts(a)

max_length = 64
vocab_size = len(tokenize.word_index) + 1
a = pad_sequences(tokenize.texts_to_sequences(a), maxlen=max_length, padding="post")

In [ ]:
a = tf.constant(a)
prediction = model.predict(a)

In [ ]:
ap = []
for i in range(len(prediction)):
    ap.append(np.argmax(prediction[i]))
np.argmax(ap)

In [ ]:
result = np.argmax(ap)
result = np.array(result).reshape(1)

In [ ]:
func.inverse_transform(result)